In [ ]:
import torch
import torch.onnx
from model import RadarCamModel

MODEL_PATH = "results/2024-06-21_23-46-23/model_8.pth"
MODEL_ONNX_PATH = "results/2024-06-21_23-46-23/model_2024-06-21_23-46-23_epoch8.onnx"

W_RESIZED = 800 
H_RESIZED = 450 
NUMBER_RADAR_POINTS = 100 # Number of radar points to use. If the number of points in a sample is less than this, they are 
                          # resampled randomly to create an array of fixed number of points. This shouldn't hurt the performance
RADAR_CHANNELS_ENCODER = [64, 128, 256, 432] 
UNET_CHANNELS_IMG = [2, 4, 8, 12, 16, 24, 32, 48] 
UNET_CHANNELS_RADAR = 24


device = "cuda" if torch.cuda.is_available() else "cpu"
 
model = RadarCamModel(RADAR_CHANNELS_ENCODER, UNET_CHANNELS_IMG, UNET_CHANNELS_RADAR).to(device)
model.eval()
model.load_state_dict(torch.load(MODEL_PATH))

#Input to the model
torch.manual_seed(0)
x1 = torch.randn(1, 1, H_RESIZED, W_RESIZED).to(device)
x2 = torch.randn(1, 3, NUMBER_RADAR_POINTS).to(device)

torch.onnx.export(model,               # model being run
                (x1, x2),                         # model input (or a tuple for multiple inputs)
                MODEL_ONNX_PATH,   # where to save the model (can be a file or file-like object)
                export_params=True,        # store the trained parameter weights inside the model file
                opset_version=16,          # the ONNX version to export the model to
                input_names = ['x_cam', 'x_rad'],   # the model's input names
                output_names = ['y_depth'], # the model's output names
                do_constant_folding=True,  # whether to execute constant folding for optimization
            )